# Version 3

# Import

In [1]:
from datetime import datetime
import os
import json

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

cleaned_test = pd.read_csv('cleaned_test.csv')
autoru = pd.read_csv('autoru_cars_2.csv')
avito = pd.read_csv('cars_avito.csv')
drom_new = pd.read_csv('drom_cars.csv')

Данные собрали с трех сайтов:

- auto.ru
- avito.ru
- drom.ru

Сначала обработаем каждый набор данных по-отдельности (включая тестовые данные).

In [2]:
autoru.head()

,Unnamed: 0,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,description,features,ownership,price,model,generation
0,0,0,525E,седан,золотистый,Бензин,1981,4,1986,механическая,2.7,125,220000.0,задний,Левый,Не требует ремонта,2,Оригинал,Unknown,Unknown,Unknown,50000.0,5 серия,II (E28)
1,1,0,520i,седан,красный,Бензин,1987,4,1991,механическая,2.0,129,300000.0,задний,Левый,Не требует ремонта,3,Оригинал,Unknown,Unknown,Unknown,60000.0,5 серия,III (E34)
2,2,0,528i,седан,синий,Бензин,1981,4,1985,механическая,2.8,184,200000.0,задний,Левый,Не требует ремонта,3,Дубликат,Unknown,Unknown,Unknown,65000.0,5 серия,II (E28)
3,3,0,316i,седан,серый,Бензин,1990,4,1992,механическая,1.6,102,300000.0,задний,Левый,Не требует ремонта,3,Дубликат,Unknown,Unknown,Unknown,68000.0,3 серия,III (E36)
4,4,0,318i,седан,зелёный,Бензин,1990,4,1994,механическая,1.8,115,350000.0,задний,Левый,Не требует ремонта,3,Дубликат,Unknown,Unknown,Unknown,70000.0,3 серия,III (E36)


Привидем к одному виду признак "features" (комплектация).

In [3]:
autoru['features'] = autoru['features'].apply(lambda x: str(x[2:-2].split("', '"))[1:-1] if x != 'Unknown' else x)

Убираем лишнии признаки.

In [4]:
autoru.drop(columns=['Unnamed: 0', 'model', 'generation'], inplace=True)

Делаем аналогичные действия для всех наборов данных

In [5]:
cleaned_test.head()

,Unnamed: 0,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,description,features,ownership,price
0,0,1,520d,седан,чёрный,дизель,2016,4,2017,автоматическая,2.0,190,158836,задний,Левый,Не требует ремонта,1,Оригинал,В РОЛЬФ Ясенево представлено более 500 автомоб...,"['[{""name"":""Безопасность"",""values"":[""Антипробу...",NaN,0
1,1,1,318d,седан,белый,дизель,2018,4,2019,автоматическая,2.0,150,10,задний,Левый,Не требует ремонта,1,Оригинал,Автомобиль из демонстрационного парка по спец ...,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",NaN,0
2,2,1,550i xDrive,седан,синий,бензин,2009,4,2012,автоматическая,4.4,407,120000,полный,Левый,Не требует ремонта,2,Оригинал,Машина собиралась под заказ! \n\n,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",7 лет и 2 месяца,0
3,3,1,30d,внедорожник,белый,дизель,2014,5,2015,автоматическая,3.0,249,111466,полный,Левый,Не требует ремонта,2,Оригинал,АВТОМОБИЛЬ В БЕЗУПРЕЧНОМ ВНЕШНЕМ И ТЕХНИЧЕСКОМ...,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",NaN,0
4,4,1,M50d,внедорожник,синий,дизель,2014,5,2019,автоматическая,3.0,381,11891,полный,Левый,Не требует ремонта,1,Оригинал,Автомобиль не имеет собственников ( физ. лиц) ...,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",NaN,0


In [6]:
cleaned_test.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
def test_features(row):
    
    if len(row) > 2:
    
        list_of_features = []

        for i in row[4:-4].split("},{"): 
            temp_dict = json.loads('{' + i + '}')
            for feature in temp_dict['values']:
                list_of_features.append(feature)                         
        return str(list_of_features)[1:-1]
    else:
        return 'Unknown'

cleaned_test['features'] = cleaned_test['features'].apply(test_features)

In [8]:
avito.head()

,Unnamed: 0,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,description,features,ownership,price
0,0,0,18d xDrive Steptronic,внедорожник,чёрный,дизель,2019,5,2020,автоматческая,2.0,150,0,полный,Левый,Новый,0,NaN,Sport Line,"[Кожа, Кожаный руль, Передних сидений, Зеркал,...",NaN,2526000
1,1,0,18d xDrive Steptronic,внедорожник,бежевый,дизель,2019,5,2020,автоматческая,2.0,150,0,полный,Левый,Новый,0,NaN,Advantage Plus,[],NaN,2990000
2,2,0,18d xDrive Steptronic,внедорожник,бежевый,дизель,2019,5,2020,автоматческая,2.0,150,0,полный,Левый,Новый,0,NaN,Advantage,[],NaN,2870000
3,3,0,218i,седан,серый,бензин,2019,4,2020,роботизированная,1.5,140,0,передний,Левый,Новый,0,NaN,M Sport,"[Ткань, Кожаный руль, Передних сидений, Руля, ...",NaN,2280990
4,4,0,218i,седан,синий,бензин,2019,4,2020,роботизированная,1.5,140,0,передний,Левый,Новый,0,NaN,M Sport,"[Ткань, Кожаный руль, Передних сидений, Руля, ...",NaN,2169120


In [9]:
avito['features'] = avito['features'].apply(lambda x: str(x[1:-2].split(', '))[1:-1] if len(x)>3 else 'Unknown')

In [10]:
avito.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
drom_new.head()

,Unnamed: 0,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,description,features,ownership,price
0,0,0,520d,седан,черный,дизель,2010,4,2013,автомат,2.0,184.0,96000.0,задний,левый,NaN,3.0,NaN,NaN,"['Нагнетатель', 'Система старт-стоп', 'Электро...",NaN,1220000
1,1,0,NaN,седан,серебристый,бензин,2017,4,2019,автомат,4.4,625.0,3600.0,полный,левый,NaN,1.0,NaN,NaN,"['Зимняя комплектация', 'Нагнетатель', 'Систем...",NaN,7600000
2,2,0,850Ci,купе,синий,бензин,1989,2,1993,автомат,5.0,300.0,160000.0,задний,левый,NaN,3.0,NaN,NaN,"['Гидроусилитель руля', 'Алюминиевые колесные ...",NaN,2150000
3,3,0,NaN,внедорожник,черный,дизель,2019,5,2019,автомат,3.0,249.0,19100.0,полный,левый,NaN,1.0,NaN,NaN,NaN,NaN,4050000
4,4,0,xDrive 30d,внедорожник,черный,дизель,2006,5,2007,автомат,3.0,235.0,203000.0,полный,левый,NaN,3.0,NaN,NaN,"['Зимняя комплектация', 'Нагнетатель', 'Поддер...",NaN,1050000


In [12]:
drom_new['features'] = drom_new['features'].apply(lambda x: str(x[2:-2].split("', '"))[1:-1] if type(x)==str else 'Unknown')

In [13]:
drom_new.drop(columns=['Unnamed: 0'], inplace=True)

In [14]:
display(autoru.info())
display(cleaned_test.info())
display(avito.info())
display(drom_new.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6536 entries, 0 to 6535
Data columns (total 21 columns):
test                   6536 non-null int64
style                  6536 non-null object
body                   6536 non-null object
color                  6536 non-null object
fuel                   6536 non-null object
model_date             6536 non-null int64
doors                  6536 non-null int64
production_date        6536 non-null int64
transmission           6536 non-null object
engine_displacement    6536 non-null float64
engine_power           6536 non-null int64
mileage                6536 non-null float64
drive_type             6536 non-null object
wheel                  6536 non-null object
condition              6536 non-null object
owners                 6536 non-null int64
vehicle_title          6536 non-null object
description            6536 non-null object
features               6536 non-null object
ownership              6536 non-null object
price            

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3837 entries, 0 to 3836
Data columns (total 21 columns):
test                   3837 non-null int64
style                  3837 non-null object
body                   3837 non-null object
color                  3837 non-null object
fuel                   3837 non-null object
model_date             3837 non-null int64
doors                  3837 non-null int64
production_date        3837 non-null int64
transmission           3837 non-null object
engine_displacement    3836 non-null float64
engine_power           3837 non-null int64
mileage                3837 non-null int64
drive_type             3837 non-null object
wheel                  3837 non-null object
condition              3837 non-null object
owners                 3837 non-null int64
vehicle_title          3837 non-null object
description            3837 non-null object
features               3837 non-null object
ownership              1753 non-null object
price              

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6215 entries, 0 to 6214
Data columns (total 21 columns):
test                   6215 non-null int64
style                  6215 non-null object
body                   6215 non-null object
color                  6215 non-null object
fuel                   6215 non-null object
model_date             6215 non-null int64
doors                  6215 non-null int64
production_date        6215 non-null int64
transmission           6215 non-null object
engine_displacement    6215 non-null float64
engine_power           6215 non-null int64
mileage                6215 non-null int64
drive_type             6215 non-null object
wheel                  6202 non-null object
condition              6215 non-null object
owners                 6215 non-null int64
vehicle_title          0 non-null float64
description            5189 non-null object
features               6215 non-null object
ownership              0 non-null float64
price                  

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406 entries, 0 to 1405
Data columns (total 21 columns):
test                   1406 non-null int64
style                  383 non-null object
body                   1406 non-null object
color                  1379 non-null object
fuel                   1402 non-null object
model_date             1406 non-null int64
doors                  1406 non-null int64
production_date        1406 non-null int64
transmission           1403 non-null object
engine_displacement    1402 non-null float64
engine_power           1392 non-null float64
mileage                1358 non-null float64
drive_type             1402 non-null object
wheel                  1404 non-null object
condition              0 non-null float64
owners                 1355 non-null float64
vehicle_title          0 non-null float64
description            0 non-null float64
features               1406 non-null object
ownership              0 non-null float64
price                 

None

После перебора различных вариантов лучший результат получился на данных auto.ru.

Объеденили тестовые данные и данные auto.ru в один датасет.

In [15]:
df = pd.concat([cleaned_test, autoru])

In [16]:
df = df.drop(['ownership', 'description'], axis=1)

In [17]:
df.isna().sum()

test                   0
style                  0
body                   0
color                  0
fuel                   0
model_date             0
doors                  0
production_date        0
transmission           0
engine_displacement    1
engine_power           0
mileage                0
drive_type             0
wheel                  0
condition              0
owners                 0
vehicle_title          0
features               0
price                  0
dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10373 entries, 0 to 6535
Data columns (total 19 columns):
test                   10373 non-null int64
style                  10373 non-null object
body                   10373 non-null object
color                  10373 non-null object
fuel                   10373 non-null object
model_date             10373 non-null int64
doors                  10373 non-null int64
production_date        10373 non-null int64
transmission           10373 non-null object
engine_displacement    10372 non-null float64
engine_power           10373 non-null int64
mileage                10373 non-null float64
drive_type             10373 non-null object
wheel                  10373 non-null object
condition              10373 non-null object
owners                 10373 non-null int64
vehicle_title          10373 non-null object
features               10373 non-null object
price                  10373 non-null float64
dtypes: float64(3), int64(6), object(10)

Привили данные к единому виду.

In [19]:
df.loc[df['body'] == 'хетчбэк', 'body'] = 'хэтчбек'
df.loc[df['body'] == 'минивэн', 'body'] = 'компактвэн'
# df.loc[df['body'] == 'купе-хардтоп', 'body'] = 'купе'

In [20]:
df.loc[df['color'] == 'зеленый', 'color'] = 'зелёный'
df.loc[df['color'] == 'золотой', 'color'] = 'золотистый'
df.loc[df['color'] == 'серебряный', 'color'] = 'серебристый'
df.loc[df['color'] == 'черный', 'color'] = 'чёрный'

In [21]:
df.loc[df['fuel'] == ' Бензин', 'fuel'] = 'бензин'
df.loc[df['fuel'] == ' Гибрид', 'fuel'] = 'гибрид'
df.loc[df['fuel'] == ' Дизель', 'fuel'] = 'дизель'
df.loc[df['fuel'] == ' Электро', 'fuel'] = 'электро'
df.loc[df['fuel'] == ' Бензин, газобаллонное оборудование', 'fuel'] = 'бензин-газ'

In [22]:
df.loc[df['transmission'] == 'автомат', 'transmission'] = 'автоматическая'
df.loc[df['transmission'] == 'автоматческая', 'transmission'] = 'автоматическая'
df.loc[df['transmission'] == 'механика', 'transmission'] = 'механическая'

In [23]:
df[df['engine_displacement'] == '170.0']

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,features,price


In [24]:
df['engine_displacement'].value_counts()

2.0      3860
3.0      3615
4.4       894
2.5       792
1.6       452
4.8       184
1.5       127
2.2        85
2.8        65
1.8        65
4.0        45
1.9        42
3.5        26
5.0        18
2.9        16
6.0        16
3.6        14
4.6        11
3.4        11
5.4         9
6.6         7
170.0       5
0.7         3
3.3         3
4.9         2
1.7         1
3.8         1
3.2         1
2.4         1
2.7         1
Name: engine_displacement, dtype: int64

In [25]:
df.loc[df['engine_displacement'] == '170.0', 'engine_power'] = 170
df.loc[df['engine_displacement'] == '170.0', 'engine_displacement'] = '3.0'

In [26]:
df.loc[df['wheel'] == 'левый', 'wheel'] = 'Левый'

# Remove duplicates

In [27]:
test = df[df['test'] == 1]
not_test = df[df['test'] == 0]

In [28]:
not_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6536 entries, 0 to 6535
Data columns (total 19 columns):
test                   6536 non-null int64
style                  6536 non-null object
body                   6536 non-null object
color                  6536 non-null object
fuel                   6536 non-null object
model_date             6536 non-null int64
doors                  6536 non-null int64
production_date        6536 non-null int64
transmission           6536 non-null object
engine_displacement    6536 non-null object
engine_power           6536 non-null int64
mileage                6536 non-null float64
drive_type             6536 non-null object
wheel                  6536 non-null object
condition              6536 non-null object
owners                 6536 non-null int64
vehicle_title          6536 non-null object
features               6536 non-null object
price                  6536 non-null float64
dtypes: float64(2), int64(6), object(11)
memory usage: 1021.2

In [29]:
not_test = not_test.drop_duplicates()

In [30]:
not_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6510 entries, 0 to 6535
Data columns (total 19 columns):
test                   6510 non-null int64
style                  6510 non-null object
body                   6510 non-null object
color                  6510 non-null object
fuel                   6510 non-null object
model_date             6510 non-null int64
doors                  6510 non-null int64
production_date        6510 non-null int64
transmission           6510 non-null object
engine_displacement    6510 non-null object
engine_power           6510 non-null int64
mileage                6510 non-null float64
drive_type             6510 non-null object
wheel                  6510 non-null object
condition              6510 non-null object
owners                 6510 non-null int64
vehicle_title          6510 non-null object
features               6510 non-null object
price                  6510 non-null float64
dtypes: float64(2), int64(6), object(11)
memory usage: 1017.2

In [31]:
df = pd.concat([not_test, test])

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10347 entries, 0 to 3836
Data columns (total 19 columns):
test                   10347 non-null int64
style                  10347 non-null object
body                   10347 non-null object
color                  10347 non-null object
fuel                   10347 non-null object
model_date             10347 non-null int64
doors                  10347 non-null int64
production_date        10347 non-null int64
transmission           10347 non-null object
engine_displacement    10346 non-null object
engine_power           10347 non-null int64
mileage                10347 non-null float64
drive_type             10347 non-null object
wheel                  10347 non-null object
condition              10347 non-null object
owners                 10347 non-null int64
vehicle_title          10347 non-null object
features               10347 non-null object
price                  10347 non-null float64
dtypes: float64(2), int64(6), object(11)


Добавляем новые признаки:

- налог
- налог на одну лошадь
- количество дней в использовании
- средний пробег за день

In [33]:
def tax(data):
    
    if data<=100:
        return float(data*12)
    
    elif data>100 and data<=125:
        return float(data*25)
    
    elif data>125 and data<=150:
        return float(data*35)
    
    elif data>150 and data<=175:
        return float(data*45)
    
    elif data>175 and data<=200:
        return float(data*50)
    
    elif data>200 and data<=225:
        return float(data*65)
    
    elif data>225 and data<=250:
        return float(data*75)
        
    elif data>250:
        return float(data*150)
    
df['tax'] = df['engine_power'].apply(tax)   

In [34]:
def tax_per_horse(data):
    
    if data<=100:
        return 12
    
    elif data>100 and data<=125:
        return 25
    
    elif data>125 and data<=150:
        return 35
    
    elif data>150 and data<=175:
        return 45
    
    elif data>175 and data<=200:
        return 50
    
    elif data>200 and data<=225:
        return 65
    
    elif data>225 and data<=250:
        return 75
        
    elif data>250:
        return 150
    
df['tax_per_horse'] = df['engine_power'].apply(tax_per_horse)

In [35]:
def days_in_use(row):
    
    # Отсчет дней будем вести от начала года даты производства, так как мы не знаем точной даты
    delta = datetime.now() - datetime(row,1,1)
    return delta.days

df['days_in_use'] = df['production_date'].apply(days_in_use)

In [36]:
df['mileage_per_day'] = df['mileage'] / df['days_in_use']

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10347 entries, 0 to 3836
Data columns (total 23 columns):
test                   10347 non-null int64
style                  10347 non-null object
body                   10347 non-null object
color                  10347 non-null object
fuel                   10347 non-null object
model_date             10347 non-null int64
doors                  10347 non-null int64
production_date        10347 non-null int64
transmission           10347 non-null object
engine_displacement    10346 non-null object
engine_power           10347 non-null int64
mileage                10347 non-null float64
drive_type             10347 non-null object
wheel                  10347 non-null object
condition              10347 non-null object
owners                 10347 non-null int64
vehicle_title          10347 non-null object
features               10347 non-null object
price                  10347 non-null float64
tax                    10347 non-null flo

In [38]:
df.to_csv('united_3.csv', index=False)